In this notebook, I will use a separate dataset to ensure the model does in fact converge in other situations (because if trained on dw ds its not converging). 

In [0]:
%cd ..
%cd root
import numpy as np
import pandas as pd
df = pd.read_csv("basic_data.csv")
df.head()
df.fillna('', inplace=True)
print(len(df))
x = df["x"].tolist()
y = df["y"].tolist()

/
/root
50330


In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()

for line in x:
    input_texts.append(str(line).split(" "))
    for word in str(line).split(" "):
        if word not in input_words:
            input_words.add(word)

for line in y:
  target_texts.append(['\t'] + str(line).split(" ") + ['\n'])
  for word in str(line).split(" "):
    if word not in target_words:
        target_words.add(word)
target_words.add("\t")
target_words.add("\n")
  
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 50330
Number of unique input tokens: 104
Number of unique output tokens: 302
Max sequence length for inputs: 3
Max sequence length for outputs: 3


In [0]:
input_texts[:5]

[['23', 'plus', '56'],
 ['52', 'subtract', '22'],
 ['90', 'plus', '13'],
 ['23', 'add', '58'],
 ['78', 'subtract', '89']]

In [0]:
target_texts[:5]

[['\t', '79', '\n'],
 ['\t', '30', '\n'],
 ['\t', '103', '\n'],
 ['\t', '81', '\n'],
 ['\t', '-11', '\n']]

In [0]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])
print(target_token_index)

{'\t': 0, '\n': 1, '-1': 2, '-10': 3, '-11': 4, '-12': 5, '-13': 6, '-14': 7, '-15': 8, '-16': 9, '-17': 10, '-18': 11, '-19': 12, '-2': 13, '-20': 14, '-21': 15, '-22': 16, '-23': 17, '-24': 18, '-25': 19, '-26': 20, '-27': 21, '-28': 22, '-29': 23, '-3': 24, '-30': 25, '-31': 26, '-32': 27, '-33': 28, '-34': 29, '-35': 30, '-36': 31, '-37': 32, '-38': 33, '-39': 34, '-4': 35, '-40': 36, '-41': 37, '-42': 38, '-43': 39, '-44': 40, '-45': 41, '-46': 42, '-47': 43, '-48': 44, '-49': 45, '-5': 46, '-50': 47, '-51': 48, '-52': 49, '-53': 50, '-54': 51, '-55': 52, '-56': 53, '-57': 54, '-58': 55, '-59': 56, '-6': 57, '-60': 58, '-61': 59, '-62': 60, '-63': 61, '-64': 62, '-65': 63, '-66': 64, '-67': 65, '-68': 66, '-69': 67, '-7': 68, '-70': 69, '-71': 70, '-72': 71, '-73': 72, '-74': 73, '-75': 74, '-76': 75, '-77': 76, '-78': 77, '-79': 78, '-8': 79, '-80': 80, '-81': 81, '-82': 82, '-83': 83, '-84': 84, '-85': 85, '-86': 86, '-87': 87, '-88': 88, '-89': 89, '-9': 90, '-90': 91, '-91': 9

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[word]] = 1.
    for t, word in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[word]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

model.save('m2_simp.h5')

Train on 40264 samples, validate on 10066 samples
Epoch 1/100
40264/40264 [==============================] - 11s 270us/step - loss: 1.6560 - accuracy: 0.3379 - val_loss: 1.4314 - val_accuracy: 0.3430
Epoch 2/100
40264/40264 [==============================] - 9s 212us/step - loss: 1.2860 - accuracy: 0.3506 - val_loss: 1.1986 - val_accuracy: 0.3504
Epoch 3/100
40264/40264 [==============================] - 9s 212us/step - loss: 1.0918 - accuracy: 0.3637 - val_loss: 1.0876 - val_accuracy: 0.3587
Epoch 4/100
40264/40264 [==============================] - 9s 212us/step - loss: 0.9727 - accuracy: 0.3752 - val_loss: 0.9277 - val_accuracy: 0.3852
Epoch 5/100
40264/40264 [==============================] - 8s 210us/step - loss: 0.8962 - accuracy: 0.3846 - val_loss: 0.9776 - val_accuracy: 0.3729
Epoch 6/100
40264/40264 [==============================] - 9s 224us/step - loss: 0.8424 - accuracy: 0.3916 - val_loss: 0.8378 - val_accuracy: 0.3914
Epoch 7/100
40264/40264 [==============================

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['23', 'plus', '56']
Decoded sentence: 79

-
Input sentence: ['52', 'subtract', '22']
Decoded sentence: 30

-
Input sentence: ['90', 'plus', '13']
Decoded sentence: 103

-
Input sentence: ['23', 'add', '58']
Decoded sentence: 81

-
Input sentence: ['78', 'subtract', '89']
Decoded sentence: -11

-
Input sentence: ['89', 'subtract', '61']
Decoded sentence: 28

-
Input sentence: ['39', 'subtract', '9']
Decoded sentence: 30

-
Input sentence: ['31', 'plus', '80']
Decoded sentence: 111

-
Input sentence: ['73', 'plus', '30']
Decoded sentence: 103

-
Input sentence: ['24', 'subtract', '42']
Decoded sentence: -18

-
Input sentence: ['14', 'add', '49']
Decoded sentence: 63

-
Input sentence: ['72', 'add', '3']
Decoded sentence: 75

-
Input sentence: ['95', 'subtract', '78']
Decoded sentence: 17

-
Input sentence: ['87', 'plus', '16']
Decoded sentence: 103

-
Input sentence: ['31', 'add', '17']
Decoded sentence: 48

-
Input sentence: ['12', 'plus', '1']
Decoded sentence: 13

-